In [68]:
import pandas as pd
import numpy as np

In [69]:
df = pd.read_csv("SIZE PRICES.csv", dtype=str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48098 entries, 0 to 48097
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Brand        47809 non-null  object
 1   Style        48098 non-null  object
 2   Colour       46341 non-null  object
 3   Size         43664 non-null  object
 4   Description  48098 non-null  object
 5   PLU          48098 non-null  object
 6   Cost         48098 non-null  object
 7   Retail       48098 non-null  object
 8   Second       48098 non-null  object
 9   Wholesale    48098 non-null  object
 10  Unnamed: 10  0 non-null      object
dtypes: object(11)
memory usage: 4.0+ MB


In [70]:
df[df['PLU'] == '0001467513']

,Brand,Style,Colour,Size,Description,PLU,Cost,Retail,Second,Wholesale,Unnamed: 10
10701,ASHWOOD,AWM-441,BLACK/NAP,8XL,BIKER STYLE,0001467513,63.00,279.00,0.00,63.00,NaN


In [71]:
df = pd.read_csv("SIZE PRICES.csv", dtype=str)
df["Parent Reference"] = df['PLU'].str[:-2]
df['Cost'] = df['Cost'].astype(np.float64)
df['Retail'] = df['Retail'].astype(np.float64)
df['Second'] = df['Second'].astype(np.float64)
df['Wholesale'] = df['Wholesale'].astype(np.float64)
df['Unnamed: 10'] = df['Unnamed: 10'].astype(np.float64)
temp1 = df[((df['Size'] == '8XL') | (df['Size'] == '9XL'))]
temp2 = df[~((df['Size'] == '8XL') | (df['Size'] == '9XL'))]
temp1 = temp1.sort_values(by='PLU', ascending=True)
temp1['Retail'] = temp1['Retail'].replace(0.0, np.NaN)
# temp1['Retail'] = temp1['Retail'].fillna(method='ffill')
temp1['Retail'] = temp1['Retail'].ffill()
df = pd.concat([temp1, temp2], ignore_index=True)
df.head()

,Brand,Style,Colour,Size,Description,PLU,Cost,Retail,Second,Wholesale,Unnamed: 10,Parent Reference
0,ASHWOOD,SAMMY,MID BROWN/APP,8XL,2 BUTTON BLAZER LAMB,0000000113,68.0,289.0,0.0,68.0,NaN,00000001
1,ASHWOOD,SAMMY,MID BROWN/APP,9XL,2 BUTTON BLAZER LAMB,0000000114,0.0,309.0,0.0,0.0,NaN,00000001
2,ASHWOOD,SAMMY,COGNAC/APP,8XL,2 BUTTON BLAZER LAMB,0000001513,68.0,289.0,0.0,68.0,NaN,00000015
3,ASHWOOD,SAMMY,COGNAC/APP,9XL,2 BUTTON BLAZER LAMB,0000001514,0.0,309.0,0.0,0.0,NaN,00000015
4,ASHWOOD,SAMMY,BLACK/NAP,8XL,2 BUTTON BLAZER LAMB,0000002913,68.0,289.0,0.0,68.0,NaN,00000029


In [72]:
vs_prices = pd.read_csv("AdjustedPricesForLargerSizes with_customs.csv", dtype=str)

In [73]:
temp = vs_prices[['customs_description', 'Parent Reference', 'Categories']]
temp = temp[~temp.isnull().any(axis=1)]
dict_customs_ref = pd.Series(temp.customs_description.values,index=temp['Parent Reference']).to_dict()
df['Categories'] = df['Parent Reference']
df['Categories'] = df['Categories'].map(dict_customs_ref)

In [74]:
list(df['Categories'].value_counts().reset_index()['Categories'])

['Ladies > Leather Jackets',
 'Mens > Leather Jackets',
 'Mens > Accessories',
 'Ladies > Handbags',
 'Ladies > Accessories',
 'Mens > Sheepskin Jackets',
 '(No assigned category)',
 'Ladies > Skirts & Jeans',
 'Sheepskin Hats',
 'Ladies > Sheepskin Jackets',
 'Mens > Man Bags & Briefcases',
 'Gift & Accessories',
 'Luggage',
 'Home & Living',
 'Ladies > Purses',
 'Ladies > Footwear',
 'Sheepskin & Fur > Footwear',
 'Ladies',
 'Mens > Footwear',
 'Mens > Wallets',
 'Sheepskin Rugs',
 'Sheepskin Earmuffs',
 'Sheepskin & Fur > Kids']

In [75]:
ladies_sizes = ['8', '10', '12', '14', '16', '18', '20', '22', '24', '26', '28', '30']
mens_sizes = ['XS', 'S', 'M', 'L', 'XL', '2XL', '3XL', '4XL', '5XL', '6XL', '7XL', '8XL', '9XL']

other_categories = ['Mens > Accessories', 'Ladies > Handbags', 'Ladies > Accessories', 'Ladies > Skirts & Jeans',
 '(No assigned category)', 'Sheepskin Hats', 'Mens > Man Bags & Briefcases', 'Gift & Accessories', 'Luggage',
 'Home & Living', 'Ladies > Purses', 'Ladies > Footwear', 'Sheepskin & Fur > Footwear', 'Ladies', 'Mens > Footwear',
 'Mens > Wallets', 'Sheepskin Rugs', 'Sheepskin Earmuffs', 'Sheepskin & Fur > Kids']

# temp = df[~df['Categories'].isna()]
# temp = temp.drop_duplicates(subset=['Parent Reference'], keep='first')
# # temp['Categories'] = 'Mens > Leather Jackets'
# temp_dict_categories_ref = pd.Series(temp['Categories'].values,index=temp['Parent Reference']).to_dict()
# temp_categories_notnull = df[~df['Categories'].isna()]
# temp_categories_null = df[df['Categories'].isna()]
# temp_categories_null['Categories'] = temp_categories_null['Parent Reference']
# temp_categories_null['Categories'] = temp_categories_null['Categories'].map(temp_dict_categories_ref)
# df = pd.concat([temp_categories_null, temp_categories_notnull], ignore_index=True)

df_vssame = df[~df['Categories'].isin(other_categories)].copy()
df_vsother = df[df['Categories'].isin(other_categories)].copy()

print(df.shape, df_vssame.shape, df_vsother.shape)

df_gr150 = df_vssame[df_vssame['Retail'] >= 100]
df_ls150 = df_vssame[df_vssame['Retail'] < 100]

mens_df_jacketgr150 = df_gr150[df_gr150['Size'].isin(mens_sizes)].copy()
ladies_df_jacketgr150 = df_gr150[df_gr150['Size'].isin(ladies_sizes)].copy()
other_df_jacketgr150 = df_gr150[(~df_gr150['Size'].isin(ladies_sizes)) & (~df_gr150['Size'].isin(mens_sizes))].copy()

mens_temp1 = mens_df_jacketgr150[mens_df_jacketgr150['Categories'].isna()]
mens_temp2 = mens_df_jacketgr150[~mens_df_jacketgr150['Categories'].isna()]
# temp = mens_temp2.drop_duplicates(subset=['Parent Reference'], keep='first')
# # temp['Categories'] = 'Mens > Leather Jackets'
# temp_dict_categories_ref = pd.Series(temp['Categories'].values,index=temp['Parent Reference']).to_dict()
# mens_temp1['Categories'] = mens_temp1['Parent Reference']
# mens_temp1['Categories'] = mens_temp1['Categories'].map(temp_dict_categories_ref)
mens_temp1['Categories'] = 'Mens > Leather Jackets'
mens_df_jacketgr150 = pd.concat([mens_temp1, mens_temp2], ignore_index=True)

ladies_temp1 = ladies_df_jacketgr150[ladies_df_jacketgr150['Categories'].isna()]
ladies_temp2 = ladies_df_jacketgr150[~ladies_df_jacketgr150['Categories'].isna()]
# temp = ladies_temp2.drop_duplicates(subset=['Parent Reference'], keep='first')
# # temp['Categories'] = 'Ladies > Leather Jackets'
# temp_dict_categories_ref = pd.Series(temp['Categories'].values,index=temp['Parent Reference']).to_dict()
# ladies_temp1['Categories'] = ladies_temp1['Parent Reference']
# ladies_temp1['Categories'] = ladies_temp1['Categories'].map(temp_dict_categories_ref)
ladies_temp1['Categories'] = 'Ladies > Leather Jackets'
ladies_df_jacketgr150 = pd.concat([ladies_temp1, ladies_temp2], ignore_index=True)

df_gr150 = pd.concat([mens_df_jacketgr150, ladies_df_jacketgr150, other_df_jacketgr150], ignore_index=True)

df_vssame = pd.concat([df_gr150, df_ls150], ignore_index=True)

df = pd.concat([df_vssame, df_vsother], ignore_index=True)
print(df.shape)

(48098, 13) (42506, 13) (5592, 13)
(48098, 13)


C:\Users\retai\AppData\Local\Temp\ipykernel_1544\2403537521.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mens_temp1['Categories'] = 'Mens > Leather Jackets'
C:\Users\retai\AppData\Local\Temp\ipykernel_1544\2403537521.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ladies_temp1['Categories'] = 'Ladies > Leather Jackets'


In [76]:
df[df['PLU'] == '0001467514']

,Brand,Style,Colour,Size,Description,PLU,Cost,Retail,Second,Wholesale,Unnamed: 10,Parent Reference,Categories
719,ASHWOOD,AWM-441,BLACK/NAP,9XL,BIKER STYLE,0001467514,0.0,299.0,0.0,0.0,NaN,00014675,Mens > Leather Jackets


In [77]:
df['Categories'].value_counts()

Categories
Mens > Leather Jackets          15831
Ladies > Leather Jackets        15264
Mens > Accessories               1337
Ladies > Handbags                1094
Ladies > Accessories              942
Mens > Sheepskin Jackets          615
(No assigned category)            300
Ladies > Skirts & Jeans           300
Sheepskin Hats                    288
Ladies > Sheepskin Jackets        277
Mens > Man Bags & Briefcases      211
Gift & Accessories                197
Luggage                           189
Home & Living                     181
Ladies > Purses                   175
Ladies > Footwear                 142
Sheepskin & Fur > Footwear        139
Ladies                             23
Mens > Footwear                    20
Mens > Wallets                     18
Sheepskin Rugs                     17
Sheepskin Earmuffs                 13
Sheepskin & Fur > Kids              6
Name: count, dtype: int64

In [78]:
mens_df = df[(df['Categories'] == 'Mens > Leather Jackets') | (df['Categories'] == 'Mens > Sheepskin Jackets')]
ladies_df = df[(df['Categories'] == 'Ladies > Leather Jackets') | (df['Categories'] == 'Ladies > Sheepskin Jackets')]
other_df = df[(df['Categories'] != 'Mens > Leather Jackets') & (df['Categories'] != 'Mens > Sheepskin Jackets') & 
              (df['Categories'] != 'Ladies > Leather Jackets') & (df['Categories'] != 'Ladies > Sheepskin Jackets')]

In [79]:
mens_size = ['2XL', '3XL', '4XL', '5XL', '6XL', '7XL', '8XL', '9XL']
print(mens_df.shape)
mens_df_2XL = mens_df[mens_df['Size'] == '2XL']
mens_dict_saleprice_title_2XL = pd.Series(mens_df_2XL['Retail'].values,index=mens_df_2XL['Parent Reference']).to_dict()
mens_df_XL = mens_df[mens_df['Size'] == 'XL']
mens_dict_saleprice_title_XL = pd.Series(mens_df_XL['Retail'].values,index=mens_df_XL['Parent Reference']).to_dict()
mens_df_L = mens_df[mens_df['Size'] == 'L']
mens_dict_saleprice_title_L = pd.Series(mens_df_L['Retail'].values,index=mens_df_L['Parent Reference']).to_dict()
mens_dict_saleprice_title = mens_dict_saleprice_title_2XL | mens_dict_saleprice_title_XL | mens_dict_saleprice_title_L
mens_df_greater18 = mens_df[mens_df['Size'].isin(mens_size)].copy()
mens_df_lesseq18 = mens_df[~mens_df['Size'].isin(mens_size)].copy()
mens_df_greater18['Retail'] = mens_df_greater18['Parent Reference']
mens_df_greater18['Retail'] = mens_df_greater18['Retail'].map(mens_dict_saleprice_title)
mens_df = pd.concat([mens_df_lesseq18, mens_df_greater18], ignore_index=True)

(16446, 13)


In [80]:
ladies_size = ['18', '20', '22', '24', '26', '28', '30']
print(ladies_df.shape)
ladies_df_18 = ladies_df[ladies_df['Size'] == '18']
dict_saleprice_title_18 = pd.Series(ladies_df_18['Retail'].values,index=ladies_df_18['Parent Reference']).to_dict()
ladies_df_10 = ladies_df[ladies_df['Size'] == '10']
dict_saleprice_title_10 = pd.Series(ladies_df_10['Retail'].values,index=ladies_df_10['Parent Reference']).to_dict()
ladies_df_8 = ladies_df[ladies_df['Size'] == '8']
dict_saleprice_title_8 = pd.Series(ladies_df_8['Retail'].values,index=ladies_df_8['Parent Reference']).to_dict()
dict_saleprice_title = dict_saleprice_title_18 | dict_saleprice_title_10 | dict_saleprice_title_8
ladies_df_greater18 = ladies_df[ladies_df['Size'].isin(ladies_size)].copy()
ladies_df_lesseq18 = ladies_df[~ladies_df['Size'].isin(ladies_size)].copy()
ladies_df_greater18['Retail'] = ladies_df_greater18['Parent Reference']
ladies_df_greater18['Retail'] = ladies_df_greater18['Retail'].map(dict_saleprice_title)
ladies_df = pd.concat([ladies_df_lesseq18, ladies_df_greater18], ignore_index=True)

(15541, 13)


In [81]:
mens_size = ['3XL', '4XL', '5XL', '6XL', '7XL', '8XL', '9XL']
mens_price_inc = [20, 40, 60, 80, 100, 120, 140]
mens_columns = ['Retail']
for i in mens_columns:
    for j in range(len(mens_size)):
        mens_df.loc[mens_df['Size']==mens_size[j], i] = mens_df[i] + mens_price_inc[j]

# mens_df_change['Sale Price (Inc VAT)'] = np.where(mens_df_change['Attribute 1 (Size)']=='3XL',mens_df_change['Sale Price (Inc VAT)'].add(20),mens_df_change['Sale Price (Inc VAT)'])

In [82]:
ladies_size = ['20', '22', '24', '26', '28', '30']
ladies_price_inc = [20, 40, 60, 80, 100, 120]
ladies_columns = ['Retail']
for i in ladies_columns:
    for j in range(len(ladies_size)):
        ladies_df.loc[ladies_df['Size']==ladies_size[j], i] = ladies_df[i] + ladies_price_inc[j]

In [83]:
final_df = pd.concat([mens_df, ladies_df, other_df], ignore_index=True)
final_df['PLU int'] = final_df['PLU'].astype(np.int64)
final_df = final_df.sort_values(by=['Brand', 'Style', 'Colour'], ascending=[True, True, True]).reset_index(drop=True)
final_df = final_df.drop(['Parent Reference', 'Categories', 'PLU int'], axis=1)
df.shape, final_df.shape

((48098, 13), (48098, 11))

In [84]:
final_df.to_csv('Fashione AdjustedPricesForLargerSizes.csv', index=False)

In [85]:
final_df[final_df['PLU'] == '0001467514']

,Brand,Style,Colour,Size,Description,PLU,Cost,Retail,Second,Wholesale,Unnamed: 10
9597,ASHWOOD,AWM-441,BLACK/NAP,9XL,BIKER STYLE,0001467514,0.0,299.0,0.0,0.0,NaN


In [86]:
dict_saleprice_title_18['00037046']

229.0

In [87]:
ladies_df_greater18[ladies_df_greater18['PLU'] == '0003704606']

,Brand,Style,Colour,Size,Description,PLU,Cost,Retail,Second,Wholesale,Unnamed: 10,Parent Reference,Categories
28436,ASHWOOD,N-12,RASPBERRY,18,mandrin biker,0003704606,0.0,229.0,0.0,0.0,NaN,00037046,Ladies > Leather Jackets
